In [1]:
import pandas as pd

df = pd.read_pickle('inspection_df.pkl')
df

,inspection_id,dph_id,date,purpose,score,inspector,printable_path,violation_count,violation_codes,inspector_notes,name
0,57344281,MTIyOTQyMTA=,2023-03-28,Routine,100,Alaina Mauldin,../../_templates/87/food_2015/_report_full.cfm...,0,[],[],OAKHURST ELEMENTARY SCHOOL
1,57323814,MTIyOTQyMTA=,2022-11-04,Routine,100,Mariam Musa,../../_templates/87/food_2015/_report_full.cfm...,0,[],[],OAKHURST ELEMENTARY SCHOOL
2,57298579,MTIyOTQyMTA=,2022-04-21,Routine,100,Mariam Musa,../../_templates/87/food_2015/_report_full.cfm...,0,[],[],OAKHURST ELEMENTARY SCHOOL
3,57269737,MTIyOTQyMTA=,2021-08-23,Routine,89,Mariam Musa,../../_templates/87/food_2015/_report_full.cfm...,1,[6-1A],[Observed TCS food item (diced tomatoes) not c...,OAKHURST ELEMENTARY SCHOOL
4,57344379,MTIyOTQyOTM=,2023-03-28,Routine,100,Carley Rhoden,../../_templates/87/food_2015/_report_full.cfm...,0,[],[],DRUID HILLS MIDDLE SCHOOL
...,...,...,...,...,...,...,...,...,...,...,...
18959,56760259,MzEzNjg1MQ==,2020-05-28,Routine,99,Bresa Daise,../../_templates/87/food_2015/_report_full.cfm...,1,[17D],[Observed JBL speaker on prep top table. All e...,Bread & Butterfly
18960,56759669,OTQ5MjM3NQ==,2020-05-12,Routine,100,Bresa Daise,../../_templates/87/food_2015/_report_full.cfm...,0,[],[],Plant Based Pizzeria- Mobile
18961,56759654,Nzk1NjQzMw==,2020-05-11,Routine,92,shirleyas chambers,../../_templates/87/food_2015/_report_full.cfm...,3,"[2-2D, 11C, 17D]","[At the time of inspection, observed no handso...",Boston Fish Supreme
18962,56759650,Nzk1Mzc1MQ==,2020-05-08,Routine,99,Richard Campbell,../../_templates/87/food_2015/_report_full.cfm...,2,"[17C, 17D]","[Ceiling not smooth and easily cleanable., Obs...",SpringHill Suites By Marriott Atlanta Perimete...


## Pandas, Use Aggregate!
- Calculate aggregated values like
    - Average violations over time (Violation Rate)
    - Time in business (years_data)
    - Running average score (int)
    - Combined all the comments (string)

In [35]:
grouped = df.groupby(['dph_id'])

# For each business entity
aggregated_rows = []
for name, group in grouped:
    grouped_comments = []
    grouped_violations = []
    date_delta = group.max(axis=0)['date'] - group.min(axis=0)['date']
    data_date_range = date_delta.days/365
    lifetime_violations = group['violation_count'].sum()
    lifetime_avg = group['score'].mean()
    for index, inspection in group.iterrows():
        if len(inspection['inspector_notes']) > 0:
            grouped_comments += inspection['inspector_notes']
        if len(inspection['violation_codes']) > 0:
            grouped_violations += inspection['violation_codes']

    # Calculate violation rate: if operating less than 1 year, violation rate = lifetime violations
    # else lifetime rate = violations/years in operation
    if data_date_range <= 1.0:
        lifetime_rate = lifetime_violations
    else:
        lifetime_rate = lifetime_violations/data_date_range

    row = {
        'dph_id': name,
        'violations': grouped_violations,
        'years_data': data_date_range,
        'lifetime_count': lifetime_violations,
        'lifetime_comments': ' '.join(grouped_comments).replace('\r\n', ''),
        'lifetime_avg': lifetime_avg,
        'violation_rate': lifetime_rate
    }
    aggregated_rows.append(row)

/tmp/ipykernel_771478/1469062488.py:4: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in grouped:


In [37]:
agg_df = pd.DataFrame.from_records(aggregated_rows)
agg_df.to_pickle('entity_aggregation.pkl')
agg_df

,dph_id,violations,years_data,lifetime_count,lifetime_comments,lifetime_avg,violation_rate
0,MTE4MTc1Mg==,"[8-2B, 11A, 15C, 4-2B, 15B, 17A, 17C, 17D, 11A...",2.230137,11,Observed the concentration of the sanitizer in...,96.500000,4.932432
1,MTE4MTc2MA==,[],0.000000,0,,100.000000,0.000000
2,MTE4MTc2Nw==,"[6-1A, 6-1C, 13A]",0.000000,3,See Attachment See Attachment See Attachment,88.000000,3.000000
3,MTE4MTcxOA==,"[2-2D, 4-2B, 15A, 6-1A]",2.460274,4,Observed Handwash sink in bar used as dumpsink...,93.333333,1.625835
4,MTE4MTcyMQ==,[],0.613699,0,,100.000000,0.000000
...,...,...,...,...,...,...,...
7837,OTQ5MzY3NA==,[],0.000000,0,,100.000000,0.000000
7838,OTQ5MzY3NQ==,"[2-2D, 4-2B, 6-2, 15C, 17C, 17C, 18, 4-2B, 17C...",2.893151,23,Observed failure to have hand soap at sushi ba...,89.200000,7.949811
7839,OTQ5MzcwNQ==,"[3-1B, 8-2B, 2-2A, 2-2E, 5-1B, 11A, 3-1B, 11A,...",1.997260,10,46F milk and 65F chicken wrap sandwich. No de...,92.500000,5.006859
7840,OTQ5MzcwNw==,"[6-1A, 6-1B, 10D, 16A, 16B]",2.569863,5,Observed various salsas and sour cream being s...,95.333333,1.945629
